In [1]:
import openpyxl
import json
import re
import copy

c:\Users\HEC\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\HEC\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\HEC\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
################Module For Functional Programing#########################
from functools import reduce
from itertools import chain
from itertools import groupby
from copy import deepcopy

curry = lambda f: lambda a, *args: (
    f(a, *args) if (len(args)) else lambda *args: f(a, *args)
)

filter, map, reduce = curry(filter), curry(map), curry(reduce)

go = lambda *args: reduce(lambda a, f: f(a), args)  ## 함수도 축약 가능 ##


def dictUpdate(dic1, dic2):
    dic1.update(dic2)
    return dic1


def dictsMerge(dics):
    res = reduce(dictUpdate, dics)
    return res


def dictDeleteKeys(dic, keys):
    for k in keys:
        del dic[k]
    return dic


def flat(a):
    if isinstance(a, list):
        for i in a:
            yield from flat(i)
    else:
        yield a


def grpBy(iter, key):
    f, grpKeys, grps = key, [], []
    sorted_ = sorted(iter, key=f)
    for key, grp_data in groupby(sorted_, key=f):
        grpKeys.append(key)
        grps.append(list(grp_data))
    return grps, grpKeys


#########################################################################


In [3]:
from openpyxl import load_workbook

def load_excel(file_path, sheet_name=None):
    # Load the workbook
    workbook = load_workbook(filename=file_path, data_only=True)

    # Select a specific worksheet
    if sheet_name:
        sheet = workbook[sheet_name]
    else:
        sheet = workbook.active  # Or use workbook['SheetName'] to select a specific sheet

    res = []
    # Read data from the worksheet
    for row in sheet.iter_rows(values_only=True):
        res.append(row)
        # print(row)



    # Close the workbook (not strictly necessary, but good practice)
    workbook.close()

    return res

# pjtFamily_sheet_data = load_excel("resources/newFL.xlsx", "5.P Family")
pjtFamily_sheet_data = load_excel("resources/A.1_플랜트건축설계팀_BIM 팀 표준 Family List_Rev.6_2024.11.07_강한호.xlsm", "5.P Family")
# pjtFamily_sheet_data = load_excel("https://henginmc6eaoutlook.sharepoint.com/:x:/s/WA00011298/EQZ9s3uraz5CoWA8Zgkzso0Bop_2HkSvuma4m72ydlqn3g?e=ZyQCQE")
str(pjtFamily_sheet_data[7])

c:\Users\HEC\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


"(True, True, None, '101 Mechanical Room, 125 Maintenance Workshop', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)"

In [4]:
def find_hdr_row(sheet_data, identifier):
    res = go(
        sheet_data,
        filter(lambda x: identifier in str(x)),
        filter(lambda x: "No" in str(x) and "Family Name" in str(x)),
        list,
    )
    
    if isinstance(res, list) and len(res)>0:
        return res[0]
    else:
        return res

In [5]:
find_hdr_row(pjtFamily_sheet_data, "▣")

('▣',
 '▣',
 'No',
 'Family Name',
 'G-WM 1',
 'G-WM 2',
 'G-WM 3',
 'G-WM 4',
 'S-WM 1',
 'Item',
 'WM No.',
 'Work Master',
 'Spec.',
 'Unit',
 'Quantity',
 '표준 산출식',
 '변경 산출식',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None)

In [6]:
def find_cat(sheet, selected_row):
    pattern = r'\b\d+\.[A-Za-z]'
    
    crnt_row = selected_row
    if sheet[crnt_row][2] and re.findall(pattern, str(sheet[crnt_row][2])):
        return sheet[crnt_row][2]
    elif crnt_row < 0:
        pass
    else:
        return find_cat(sheet, crnt_row - 1)

In [7]:
def find_parent_type(sheet, selected_row):
    pattern = r'\b\d+\.\d+'
    
    crnt_row = selected_row
    if sheet[crnt_row][2] and re.findall(pattern, str(sheet[crnt_row][2])):
        return sheet[crnt_row][3]
    elif crnt_row < 0:
        pass
    else:
        return find_parent_type(sheet, crnt_row - 1)

In [8]:
# modi_sheet_data = []
# for idx,row in enumerate(pjtFamily_sheet_data):
#     newRow = list(row[:])
#     del newRow[2]
#     newRow.insert(2, find_cat(pjtFamily_sheet_data, idx))
#     modi_sheet_data.append(newRow)
# modi_sheet_data



In [9]:
def create_typeWM_match_dict_forPjt(sheet_data, hdrs=None):
    if not hdrs:
        hdrs = find_hdr_row(sheet_data, "▣")
    hdr_idx_of_FamilyName = hdrs.index("Family Name")
    
    modi_sheet_data = []
    for idx,row in enumerate(pjtFamily_sheet_data):
        newRow = list(row[:])
        # del newRow[2]
        # newRow.insert(2, find_cat(pjtFamily_sheet_data, idx))
        newRow.append(find_cat(pjtFamily_sheet_data, idx))
        newRow.append(find_parent_type(pjtFamily_sheet_data, idx))
        modi_sheet_data.append(newRow)
    
    onlyEffRows = go(
        modi_sheet_data,
        filter(lambda x: "True" in str(x)),
        list,
    )
    
    dics_keys = [
        "Item",
        "WM No.",
        "Work Master",
        "Spec.",
        "Unit",
        "표준 산출식",
        "변경 산출식",
    ]
    
    res = []
    tmp = None
    for row_idx, row_cells in enumerate(onlyEffRows):
        cat = row_cells[-2]
        parent_type = row_cells[-1]
        if "True, True" in str(row_cells):
            if tmp:
                for i in tmp:
                    res.append(i)
            tmp = []
            for famType in row_cells[hdr_idx_of_FamilyName].split(","):
                tmp.append(
                    {
                    famType.strip(): {
                        "cat": cat,
                        "parent_type": parent_type,
                        "wm_list": []
                        
                        }
                    }
                )
        elif "True, True" not in str(row_cells):
            
            # famType_names = list(tmp.keys())
            # for famType in famType_names:
            for i in tmp:
                famType = list(i.keys())[0]
                wmDic = {}
                for x in dics_keys:
                    wmDic.update({x: row_cells[hdrs.index(x)]})
                i[famType]["wm_list"].append(wmDic)
            
    return res

In [10]:
pjt_typeWM_match_dicts = create_typeWM_match_dict_forPjt(pjtFamily_sheet_data)
# effGrps[4].keys()
# pjt_typeWM_match_dicts[49]
pjt_typeWM_match_dicts

[{'101 Mechanical Room': {'cat': None,
   'parent_type': 'Room Name ',
   'wm_list': [{'Item': 'Trowel',
     'WM No.': 'A04AS173-00001',
     'Work Master': 'A04AS173-00001 Finishing Work Exterior/Interior Finish Work Steel Trowel Finish        ',
     'Spec.': 0,
     'Unit': 'M2',
     '표준 산출식': '=A',
     '변경 산출식': None},
    {'Item': 'Hardener',
     'WM No.': 'A04AS173-00003',
     'Work Master': 'A04AS173-00003 Finishing Work Exterior/Interior Finish Work Steel Trowel Finish Hardener Finish(Powder Type)       ',
     'Spec.': 0,
     'Unit': 'M2',
     '표준 산출식': '=A',
     '변경 산출식': None},
    {'Item': 'Paint-EP',
     'WM No.': 'A04AM080-00006',
     'Work Master': 'A04AM080-00006 Finishing Work Painting Work Floor Painting Epoxy Paint       ',
     'Spec.': 0,
     'Unit': 'M2',
     '표준 산출식': '=A',
     '변경 산출식': None},
    {'Item': 'Paint-EP',
     'WM No.': 'A04AM081-00007',
     'Work Master': 'A04AM081-00007 Finishing Work Painting Work Skirt Painting Epoxy Paint       H<

In [11]:
# stdFamily_sheet_data = load_excel("resources/newFL.xlsx", "3.S Family")
stdFamily_sheet_data = load_excel("resources/A.1_플랜트건축설계팀_BIM 팀 표준 Family List_Rev.6_2024.11.07_강한호.xlsm", "3.S Family")
str(stdFamily_sheet_data[3])

"(None, None, None, None, '*하기 기본 Family Name은 변경 불가 : _(Under bar), - (Dash), 띄어쓰기 변경 불가', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', None, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', None, None, None, None, None, None, None, None)"

In [12]:
# find_hdr_row(stdFamily_sheet_data, "Description")
# find_hdr_row(stdFamily_sheet_data, "'#', '#', '#'")

In [13]:
def create_typeWM_match_dict_forStd(sheet_data, hdrs=None):
    def filter_hdr_row(row, rng_idx=None):
        res_ = []
        for hdr_idx, hdr in enumerate(row):
            # if hdr!=None and hdr!='-':
            res_.append({hdr: hdr_idx})
        if rng_idx:
            res = res_[rng_idx[0]:rng_idx[1]+1]
        else:
            res = res_
        filter_res = list(filter(lambda hdr: list(hdr.keys())[0]!=None and list(hdr.keys())[0]!='-', res))
        return filter_res
    
    if not hdrs:
        common_hdrs = go(
            sheet_data,
            lambda x: find_hdr_row(x, "Description"),
            lambda x: filter_hdr_row(x, [0,11]),
        )
    # hdr_idx_of_FamilyName = common_hdrs.index("Family Name_Basic (변경 불가)")
    
    def find_hdrs_forType(common_hdrs, row):
        type_hdrs_formula = filter_hdr_row(row, rng_idx=[12,32])
        type_hdrs_paramDict = filter_hdr_row(row, rng_idx=[32,100])
        
        return {
            "표준헤더": common_hdrs,
            "산출식헤더": type_hdrs_formula,
            "매개변수헤더": type_hdrs_paramDict,
        }
        
    header_check = []
    # res = []
    
    res = {}
    type_hdrs=None
    for idx, row in enumerate(sheet_data):
        if row[2]=="#":
            cat = row[3]
            type_hdrs = find_hdrs_forType(common_hdrs, row)
            # header_check.append(find_hdrs_forType(common_hdrs, row))
        else:
            dic = {}
            if type_hdrs:
                for hdr_dic in type_hdrs["표준헤더"]:
                    dic.update(
                        {
                            list(hdr_dic.keys())[0]: row[ list(hdr_dic.values())[0] ],
                        }
                    )
                form_dic = {}
                for hdr_formulas in type_hdrs["산출식헤더"]:
                    if row[ list(hdr_formulas.values())[0] ] and row[ list(hdr_formulas.values())[0] ]!='-':
                        form_dic.update(
                            {
                                list(hdr_formulas.keys())[0]: row[ list(hdr_formulas.values())[0] ],
                            }
                        )
                
                param_dic = {}
                for hdr_params in type_hdrs["매개변수헤더"]:
                    if row[ list(hdr_params.values())[0] ] and row[ list(hdr_params.values())[0] ]!='-':
                        param_dic.update(
                            {
                                list(hdr_params.keys())[0]: row[ list(hdr_params.values())[0] ],
                            }
                        )

                dic.update(
                    {
                        "formulas": form_dic,
                        "params": param_dic,
                    }
                )
                famName = dic["Family Name_Basic (변경 불가)"]
                res.update(
                    {
                        famName : dic
                    }
                )
    
    return res

In [14]:
std_typeWM_match_dicts = create_typeWM_match_dict_forStd(stdFamily_sheet_data)
std_typeWM_match_dicts

{'Room Name ': {'No': 0.1,
  'Family Name_Basic (변경 불가)': 'Room Name ',
  'Description': None,
  'G-WM 1': None,
  'G-WM 2': None,
  'G-WM 3': None,
  'G-WM 4': None,
  'S-WM 1': None,
  '표준물량 산출식': 'Q0.1',
  'formulas': {'Floor': '=A',
   'Skirt': '=P',
   'Wall': '=P*H',
   'Ceiling': '=A',
   'Floor\n*구체제외한 모든 마감\n(Screed, Trowel, 방수,Tile, Paint)': 'Wall\n*벽체제외한 모든 마감\n(내부Plaster, Tile, 방수, Paint)'},
  'params': {'A': 'Area', 'P': 'Perimeter', 'H': 'H_Room H or CH'}},
 'H_FL_AR S17_RC Slab': {'No': 1.1,
  'Family Name_Basic (변경 불가)': 'H_FL_AR S17_RC Slab',
  'Description': '*RC Roof Slab 포함\n*SOG는 Foundation Family 적용',
  'G-WM 1': 'RC-AG',
  'G-WM 2': '-',
  'G-WM 3': '-',
  'G-WM 4': '-',
  'S-WM 1': '-',
  '표준물량 산출식': 'Q1.1',
  'formulas': {'Concrete': '=V', 'Form1': '=A', 'Rebar': '=V*RE'},
  'params': {'A': 'Area', 'V': 'Volume', 'RE': 0.1}},
 'H_FL_AR S17_RC Slab_Deck': {'No': 1.2,
  'Family Name_Basic (변경 불가)': 'H_FL_AR S17_RC Slab_Deck',
  'Description': None,
  'G-WM 1': 'R

In [15]:
pjt_typeWM_match_dicts

[{'101 Mechanical Room': {'cat': None,
   'parent_type': 'Room Name ',
   'wm_list': [{'Item': 'Trowel',
     'WM No.': 'A04AS173-00001',
     'Work Master': 'A04AS173-00001 Finishing Work Exterior/Interior Finish Work Steel Trowel Finish        ',
     'Spec.': 0,
     'Unit': 'M2',
     '표준 산출식': '=A',
     '변경 산출식': None},
    {'Item': 'Hardener',
     'WM No.': 'A04AS173-00003',
     'Work Master': 'A04AS173-00003 Finishing Work Exterior/Interior Finish Work Steel Trowel Finish Hardener Finish(Powder Type)       ',
     'Spec.': 0,
     'Unit': 'M2',
     '표준 산출식': '=A',
     '변경 산출식': None},
    {'Item': 'Paint-EP',
     'WM No.': 'A04AM080-00006',
     'Work Master': 'A04AM080-00006 Finishing Work Painting Work Floor Painting Epoxy Paint       ',
     'Spec.': 0,
     'Unit': 'M2',
     '표준 산출식': '=A',
     '변경 산출식': None},
    {'Item': 'Paint-EP',
     'WM No.': 'A04AM081-00007',
     'Work Master': 'A04AM081-00007 Finishing Work Painting Work Skirt Painting Epoxy Paint       H<

In [16]:
def insert_paramDic_toPjt_typeWM_match_dicts(pjt_typeWM_match_dicts, std_typeWM_match_dicts):
    for dic in pjt_typeWM_match_dicts:
        type_name = list(dic.keys())[0]
        parent_typeName = list(dic.values())[0]["parent_type"]
        paramDic = std_typeWM_match_dicts[parent_typeName]["params"]
        dic[type_name].update(
            {
                "params": paramDic
            }
        )
        # print(dic)
    return pjt_typeWM_match_dicts

In [17]:
insert_paramDic_toPjt_typeWM_match_dicts(pjt_typeWM_match_dicts, std_typeWM_match_dicts)

[{'101 Mechanical Room': {'cat': None,
   'parent_type': 'Room Name ',
   'wm_list': [{'Item': 'Trowel',
     'WM No.': 'A04AS173-00001',
     'Work Master': 'A04AS173-00001 Finishing Work Exterior/Interior Finish Work Steel Trowel Finish        ',
     'Spec.': 0,
     'Unit': 'M2',
     '표준 산출식': '=A',
     '변경 산출식': None},
    {'Item': 'Hardener',
     'WM No.': 'A04AS173-00003',
     'Work Master': 'A04AS173-00003 Finishing Work Exterior/Interior Finish Work Steel Trowel Finish Hardener Finish(Powder Type)       ',
     'Spec.': 0,
     'Unit': 'M2',
     '표준 산출식': '=A',
     '변경 산출식': None},
    {'Item': 'Paint-EP',
     'WM No.': 'A04AM080-00006',
     'Work Master': 'A04AM080-00006 Finishing Work Painting Work Floor Painting Epoxy Paint       ',
     'Spec.': 0,
     'Unit': 'M2',
     '표준 산출식': '=A',
     '변경 산출식': None},
    {'Item': 'Paint-EP',
     'WM No.': 'A04AM081-00007',
     'Work Master': 'A04AM081-00007 Finishing Work Painting Work Skirt Painting Epoxy Paint       H<